In [57]:
import numpy as np
import torch
np.random.seed(0)
torch.random.manual_seed(0)

## Difference from tensorflow implementation(so far)
1. Time step is not input to neural net
2. Initial state is initialized randomly and learnable, instead of zero

In [4]:
def generate_left_prob(held_out_range = [[0.1, 0.2], [0.3, 0.4]]):
    def check_heldout(left_prob):
        inrange = False
        for low, high in held_out_range:
            if left_prob>low and left_prob<high:
                inrange = True
        return inrange
    left_prob = -1
    while left_prob<0 or left_prob>0.5 or check_heldout(left_prob):
        left_prob = np.random.uniform(0, 0.5)
    return left_prob

def get_trial(current_state, left_prob, action): # get next trial after action
    assert 0<=left_prob and left_prob<=0.5, "probability out of range"
    right_prob = 0.5-left_prob
    current_state = current_state.copy()
    current_state[action] = 0
    if np.random.uniform()<left_prob:
        current_state[0] = 1
    if np.random.uniform()<right_prob:
        current_state[1] = 1
    return current_state



In [22]:
minimum = 30
for i in range(10000):
    minimum = min(minimum, np.random.uniform())

In [60]:
class A3C(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.init_h = torch.nn.Parameter(torch.zeros(1, 48))
        self.init_c = torch.nn.Parameter(torch.zeros(1, 48))
        self.lstm = torch.nn.LSTMCell(3, 48)
        self.action_head = torch.nn.Linear(48, 2)
        self.value_head = torch.nn.Linear(48, 1)
    def forward(self, prev_action, prev_reward, hidden):
        input_vec = torch.zeros(3)
        if prev_action!=-1: # if not first trial
            input_vec[prev_action] = 1
        input_vector[2] = prev_reward
        if hidden == None:
            lstm_out = self.lstm(input_vector[None, ...], (self.init_h, self.init_c))
        else:
            lstm_out = self.lstm(input_vector[None, ...], hidden)
        action = self.action_head(lstm_out)
        value = self.value_head(lstm_out)
        return action, value

agent = A3C()
optimizer = torch.optim.RMSprop(agent.parameters(), 0.007)

In [61]:
def run_block(train = True):
    num_trials = np.random.randint(50, 101)
    left_prob = generate_left_prob()
    initial_state = get_trial([0, 0], left_prob, 0)
    action = -1
    reward = 0
    hidden = None
    value_preds = []
    rewards = []
    actions = []
    for trial_num in range(num_trials):
        action_prop, value_pred = agent(action, reward, hidden)
        value_preds.append(value_pred)

SyntaxError: invalid syntax (<ipython-input-61-1992b7a26b06>, line 13)